In [1]:
import os

In [2]:
PROJECT_ID = 'qwiklabs-gcp-81966e9357970fe8' # CHANGE
BUCKET = 'qwiklabs-gcp-81966e9357970fe8' # CHANGE
MODEL_BASE = 'taxi_trained/export/exporter'
MODEL_PATH = os.path.join(MODEL_BASE,os.listdir(MODEL_BASE)[-1])
MODEL_NAME = 'taxifare'
VERSION_NAME = 'v1'

# Deploy for Online Prediction

To get our predictions we need to fetch the latest traffic information from BigQuery. Only then can we invoke our tensorflow model.

To do this we'll take advantage of [AI Platforms Custom Prediction Routines](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines) which allows us to execute custom python code in response to every online prediction request. There are 5 steps to creating a custom prediction routine:

1. Upload Model Artifacts to GCS
2. Implement Predictor interface 
3. Package the prediction code and dependencies
4. Deploy
5. Invoke API

## 1. Upload Model Artifacts to GCS

Here we upload our model weights so that AI Platform can access them.

In [19]:
!gsutil cp -r $MODEL_PATH/* gs://$BUCKET/taxifare/model/

Copying file://taxi_trained/export/exporter/1564519940/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://taxi_trained/export/exporter/1564519940/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://taxi_trained/export/exporter/1564519940/variables/variables.data-00000-of-00002 [Content-Type=application/octet-stream]...
Copying file://taxi_trained/export/exporter/1564519940/variables/variables.data-00001-of-00002 [Content-Type=application/octet-stream]...
-
Operation completed over 4 objects/78.9 KiB.                                     


## 2. Implement Predictor Interface

Interface Spec: https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines#predictor-class

This tells AI Platform how to load the model artifacts, and is where we specify our custom prediction code.

In [3]:
%%writefile predictor.py
import tensorflow as tf
from google.cloud import bigquery

class TaxifarePredictor(object):
    def __init__(self, predict_fn):
      self.predict_fn = predict_fn
    
    
    def predict(self, instances, **kwargs):
        bq = bigquery.Client()
        query_string = """
        SELECT
          *
        FROM
          `taxifare.traffic_realtime`
        ORDER BY
          time DESC
        LIMIT 1
        """
        trips = bq.query(query_string).to_dataframe()['trips_last_5min'][0]
        instances['trips_last_5min'] = [trips for _ in range(len(list(instances.items())[0][1]))]
        predictions = self.predict_fn(instances)
        return predictions
    

    @classmethod
    def from_path(cls, model_dir):
        predict_fn = tf.contrib.predictor.from_saved_model(model_dir,'predict')
        return cls(predict_fn)

Overwriting predictor.py


### Test Predictor Class Works Locally

In [4]:
import predictor

instances = {'dayofweek' : [6,7],
             'hourofday' : [12,11],
             'pickuplon' : [-73.99,-73.99], 
             'pickuplat' : [40.758,40.758],
             'dropofflat' : [40.742,40.758],
             'dropofflon' : [-73.97,-73.97]}

predictor = predictor.TaxifarePredictor.from_path(MODEL_PATH)
predictor.predict(instances)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from taxi_trained/export/exporter/1564519940/variables/variables


{'predictions': array([[7.3234653],
        [6.110935 ]], dtype=float32)}

## 3. Package Predictor Class and Dependencies

We must package the predictor as a tar.gz source distribution package.

In [5]:
%%writefile setup.py
from setuptools import setup

setup(
    name='taxifare_custom_predict_code',
    version='0.1',
    scripts=['predictor.py'])

Overwriting setup.py


In [6]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
writing taxifare_custom_predict_code.egg-info/PKG-INFO
writing top-level names to taxifare_custom_predict_code.egg-info/top_level.txt
writing dependency_links to taxifare_custom_predict_code.egg-info/dependency_links.txt
reading manifest file 'taxifare_custom_predict_code.egg-info/SOURCES.txt'
writing manifest file 'taxifare_custom_predict_code.egg-info/SOURCES.txt'
running check


creating taxifare_custom_predict_code-0.1
creating taxifare_custom_predict_code-0.1/taxifare_custom_predict_code.egg-info
copying files to taxifare_custom_predict_code-0.1...
copying README.md -> taxifare_custom_predict_code-0.1
copying predictor.py -> taxifare_custom_predict_code-0.1
copying setup.py -> taxifare_custom_predict_code-0.1
copying taxifare_custom_predict_code.egg-info/PKG-INFO -> taxifare_custom_predict_code-0.1/taxifare_custom_predict_code.egg-info
copying taxifare_custom_predict_code.egg-info/SOURCES.txt -> taxifare_custom_predict_code-0.1/taxifare_custom_predic

In [7]:
!gsutil cp dist/taxifare_custom_predict_code-0.1.tar.gz gs://$BUCKET/taxifare/predict_code/

Copying file://dist/taxifare_custom_predict_code-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


## 4. Deploy

This is similar to how we deploy standard models to AI Platform, with a few extra command line arguments.

*Warning: If you get a GCS access error, grant the 'Storage Object Viewer' role on the bucket that contains your artifacts to the service account being used.*

In [ ]:
!gcloud ai-platform models create $MODEL_NAME --regions us-central1 --enable-logging

!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin gs://$BUCKET/taxifare/model \
  --runtime-version 1.14 \
  --python-version 3.5 \
  --package-uris gs://$BUCKET/taxifare/predict_code/taxifare_custom_predict_code-0.1.tar.gz \
  --prediction-class predictor.TaxifarePredictor

ERROR: (gcloud.ai-platform.models.create) Resource in project [qwiklabs-gcp-81966e9357970fe8] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Creating version (this might take a few minutes)......⠧

## 5. Invoke API

In [30]:
import googleapiclient.discovery

instances = {'dayofweek' : [6], 
             'hourofday' : [12],
             'pickuplon' : [-73.99], 
             'pickuplat' : [40.758],
             'dropofflat' : [40.742],
             'dropofflon' : [-73.97]}

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

RuntimeError: Prediction failed: unknown error.